In [1]:
# NBA Free Agent Machine

# 1. Install, import packages and set parameters
# ! pip install get_nba_data
# ! pip install nba_py
# ! pip install nba_api
# ! pip install sklearn
# ! pip install hide_code
# ! jupyter nbextension install --py hide_code
# ! jupyter nbextension enable --py hide_code
# ! jupyter serverextension enable --py hide_code
# ! pip install hide_code --upgrade
# ! jupyter nbextension install --py hide_code
import requests
import json
import pandas as pd
import numpy as np
import nba_api
import sklearn
import ipywidgets as widgets
from ipywidgets import interact
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# 2 Data Gathering and Manipulation
# 2.1 Connect to nba_api
from nba_api.stats.endpoints import leaguedashplayerstats

# 2.2 Gather NBA player data from 2018-19 season, 2017-18 season, and 2016-17 sesaon
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2018-19')
catstats2018_2019 = playerinfo.get_data_frames()
catstats2018_2019 = pd.concat(catstats2018_2019)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2017-18')
catstats2017_2018 = playerinfo.get_data_frames()
catstats2017_2018 = pd.concat(catstats2017_2018)
playerinfo = leaguedashplayerstats.LeagueDashPlayerStats(per_mode_detailed = 'PerGame', season = '2016-17')
catstats2016_2017 = playerinfo.get_data_frames()
catstats2016_2017 = pd.concat(catstats2016_2017)

# 2.3 Remove unnecessary columns, add year to end of each column name
final2018_2019 = catstats2018_2019[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','GP','MIN','FGM','FGA',
'FG3M','FG3A','FTM','FTA','OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2018_2019 = final2018_2019.add_suffix('_2018')
final2017_2018 = catstats2017_2018[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2017_2018 = final2017_2018.add_suffix('_2017')
final2016_2017 = catstats2016_2017[['PLAYER_ID','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
 'OREB','DREB','AST','STL','BLK','PF','TOV','PTS']]
final2016_2017 = final2016_2017.add_suffix('_2016')

# 2.4 Merge all dataframes to make one master dataframe with all 3 years of data
final = final2018_2019.merge(final2017_2018, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2017', how='left')
final = final.merge(final2016_2017, left_on='PLAYER_ID_2018', right_on='PLAYER_ID_2016', how='left')

# 2.5 Add new columns for FT made and missed, FG3 made and missed, FG2 made and missed
final['FTMiss_2018'] = final.apply(lambda row: row.FTA_2018 - row.FTM_2018, axis=1)
final['FTMiss_2017'] = final.apply(lambda row: row.FTA_2017 - row.FTM_2017, axis=1)
final['FTMiss_2016'] = final.apply(lambda row: row.FTA_2016 - row.FTM_2016, axis=1)
final['FG3Miss_2018'] = final.apply(lambda row: row.FG3A_2018 - row.FG3M_2018, axis=1)
final['FG3Miss_2017'] = final.apply(lambda row: row.FG3A_2017 - row.FG3M_2017, axis=1)
final['FG3Miss_2016'] = final.apply(lambda row: row.FG3A_2016 - row.FG3M_2016, axis=1)
final['FG2Made_2018'] = final.apply(lambda row: row.FGM_2018 - row.FG3M_2018, axis=1)
final['FG2A_2018'] = final.apply(lambda row: row.FGA_2018 - row.FG3A_2018, axis=1)
final['FG2Miss_2018'] = final.apply(lambda row: row.FG2A_2018 - row.FG2Made_2018, axis=1)
final['FG2Made_2017'] = final.apply(lambda row: row.FGM_2017 - row.FG3M_2017, axis=1)
final['FG2A_2017'] = final.apply(lambda row: row.FGA_2017 - row.FG3A_2017, axis=1)
final['FG2Miss_2017'] = final.apply(lambda row: row.FG2A_2017 - row.FG2Made_2017, axis=1)
final['FG2Made_2016'] = final.apply(lambda row: row.FGM_2016 - row.FG3M_2016, axis=1)
final['FG2A_2016'] = final.apply(lambda row: row.FGA_2016 - row.FG3A_2016, axis=1)
final['FG2Miss_2016'] = final.apply(lambda row: row.FG2A_2016 - row.FG2Made_2016, axis=1)

# 2.6 Drop unnecessary columns and rename
final.drop(['FG3A_2018', 'FG2A_2018', 'FTA_2018', 'FGM_2018', 'FGA_2018',
            'FG3A_2017', 'FG2A_2017', 'FTA_2017', 'FGM_2017', 'FGA_2017', 
            'FG3A_2016', 'FG2A_2016', 'FTA_2016', 'FGM_2016', 'FGA_2016', 
            'PLAYER_ID_2017', 'PLAYER_ID_2016'], axis=1, inplace=True)
final = final.rename(index=str,columns={'PLAYER_ID_2018': 'PLAYER_ID',
                                        'PLAYER_NAME_2018':'PLAYER',
                                        'TEAM_ABBREVIATION_2018':'TEAM',
                                        'FTM_2018':'FTMade_2018', 'FTM_2017':'FTMade_2017', 'FTM_2016':'FTMade_2016',
                                        'FG3M_2018':'FG3Made_2018', 'FG3M_2017':'FG3Made_2017', 'FG3M_2016':'FG3Made_2016'})

# 2.7 Use nba_api to gather player bio information
from nba_api.stats.endpoints import leaguedashplayerbiostats
playerinfo = leaguedashplayerbiostats.LeagueDashPlayerBioStats()
PlayerBios = playerinfo.get_data_frames()
PlayerBios = pd.concat(PlayerBios)

# 2.8 Remove unnecessary columns, change the datatype of Draft to integer, merge with the final dataframe
PlayerBios = PlayerBios[['PLAYER_ID','AGE','PLAYER_HEIGHT_INCHES','PLAYER_WEIGHT','COUNTRY','DRAFT_NUMBER']]
PlayerBios['DRAFT_NUMBER'] = PlayerBios.DRAFT_NUMBER.replace('Undrafted','0')
PlayerBios['DRAFT_NUMBER'] = PlayerBios['DRAFT_NUMBER'].astype(str).astype(int)
PlayerBios = PlayerBios.rename(index=str,columns={'DRAFT_NUMBER': 'DRAFT'})
final = final.merge(PlayerBios, left_on='PLAYER_ID', right_on='PLAYER_ID', how='left')

# 2.9 Create CSV files for 2018 salaries, 2019 salaries, 2019 Free Agents with data from Spotrac.com
# 2.9.1 Read csv for 2018 NBA player salaries, merge with final dataframe
Sals2018 = pd.read_csv('2018Sals.csv', dtype={'CAP HIT':float})
Sals2018 = Sals2018.rename(index=str,columns={'CAP HIT': 'PREV_SAL'})
final = final.merge(Sals2018, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.9.2 Read csv for 2019 NBA player salaries, merge with final dataframe to create Signed Players dataframe
Sals2019 = pd.read_csv('2019Sals.csv', dtype={'CAP HIT':float})
Sals2019 = Sals2019.rename(index=str,columns={'CAP HIT': 'SAL'})
Signed = final.merge(Sals2019, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.9.3 Read csv for 2019 NBA Free Agents, merge with final dataframe to create Free Agents dataframe
FAs2019 = pd.read_csv('FAs2019.csv')
FAs2019 = FAs2019.rename(index=str,columns={'TEAM': 'PREV_TEAM'})
FAs = final.merge(FAs2019, left_on='PLAYER', right_on='PLAYER', how='inner')

# 2.10 Make the categorical variables dummy variables
# 2.10.1 If the Country is USA, change the variable to 1, if not then 0
Signed.loc[Signed['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
Signed.loc[Signed['COUNTRY'] == 'USA', 'COUNTRY'] = '1'
FAs.loc[FAs['COUNTRY'] != 'USA', 'COUNTRY'] = '0'
FAs.loc[FAs['COUNTRY'] == 'USA', 'COUNTRY'] = '1'

# 2.10.2 Assign the number for each position (PG-1,SG-2,SF-3,PF-4,C-5)
Signed.loc[Signed['POS'] == 'PG', 'POS'] = '1'
Signed.loc[Signed['POS'] == 'SG', 'POS'] = '2'
Signed.loc[Signed['POS'] == 'SF', 'POS'] = '3'
Signed.loc[Signed['POS'] == 'PF', 'POS'] = '4'
Signed.loc[Signed['POS'] == 'C', 'POS'] = '5'
FAs.loc[FAs['POS'] == 'PG', 'POS'] = '1'
FAs.loc[FAs['POS'] == 'SG', 'POS'] = '2'
FAs.loc[FAs['POS'] == 'SF', 'POS'] = '3'
FAs.loc[FAs['POS'] == 'PF', 'POS'] = '4'
FAs.loc[FAs['POS'] == 'C', 'POS'] = '5'

# 2.11 Change datatypes to perform regression
Signed['COUNTRY'] = Signed.COUNTRY.astype(np.int64)
Signed['POS'] = Signed.POS.astype(np.int64)
Signed['DRAFT'] = Signed.DRAFT.astype(np.int64)
Signed['PLAYER_WEIGHT'] = Signed.PLAYER_WEIGHT.astype(np.int64)
FAs['COUNTRY'] = FAs.COUNTRY.astype(np.int64)
FAs['POS'] = FAs.POS.astype(np.int64)
FAs['DRAFT'] = FAs.DRAFT.astype(np.int64)
FAs['PLAYER_WEIGHT'] = FAs.PLAYER_WEIGHT.astype(np.int64)

# 2.12 Fill NA values with 0
Signed = Signed.fillna(0)
FAs = FAs.fillna(0)

# 2.13 Reorder Columns for Analysis
Signedcols = Signed.columns.tolist()
Signedcols = Signedcols[:3] + Signedcols[54:57] + Signedcols[3:54] + Signedcols[57:59]
Signed = Signed[Signedcols]
FAscols = FAs.columns.tolist()
FAscols = FAscols[:3] + FAscols[58:] + FAscols[54:57] + FAscols[3:54] + FAscols[57:58]
FAs = FAs[FAscols]

# 3 Random Forest Regression
# 3.1 Importing the dataset
X_train = Signed.iloc[:, 3:58].values
y_train = Signed.iloc[:, 58].values
X_test = FAs.iloc[:, 6:].values

# 3.2 Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 0)
regressor.fit(X_train, y_train)

# 3.3 Predicting a new result
y_pred = regressor.predict(X_test)
results = pd.DataFrame(FAs['PLAYER'])
results['y_pred'] = y_pred
results['y_pred'] = results.y_pred.astype(np.int64)
FAs['PRED SAL'] = results['y_pred']

# 3.4 Set quantiles for confidence regression
QUANTILES = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# 3.5 Create original dataframe for function
preds = pd.DataFrame(columns = ['method', 'q', 'x'])

# 3.6 Quantile Forest Regression code from https://colab.research.google.com/
# drive/1nXOlrmVHqCHiixqiMF6H8LSciz583_W2#scrollTo=G0lU1DPizHpO&forceEdit=true&offline=true&sandboxMode=true
def rf_quantile(m, X, q):
    rf_preds = []
    for estimator in m.estimators_:
        rf_preds.append(estimator.predict(X))
    rf_preds = np.array(rf_preds).transpose()  # One row per record.
    return np.percentile(rf_preds, q * 100, axis=1)

preds.loc[preds.method == 'Random forests', 'pred'] = np.concatenate(
    [rf_quantile(regressor, X_test, q) for q in QUANTILES])

# 3.7 Create dataframe for each confidence level
zero_pct = preds.iloc[0:153]['pred']
ten_pct = preds[153:306]['pred']
twenty_pct = preds[306:459]['pred']
thirty_pct = preds[459:612]['pred']
forty_pct = preds[612:765]['pred']
fifty_pct = preds[765:918]['pred']
sixty_pct = preds[918:1071]['pred']
seventy_pct = preds[1071:1224]['pred']
eighty_pct = preds[1224:1377]['pred']
ninety_pct = preds[1377:1530]['pred']
one_hundred_pct = preds[1530:]['pred']

# 3.8 Add all dataframes to new dataframe with each free agent
pct_likelihood = FAs[['PLAYER']]
pct_likelihood['zero'] = zero_pct.reset_index(drop=True)
pct_likelihood['ten'] = ten_pct.reset_index(drop=True)
pct_likelihood['twenty'] = twenty_pct.reset_index(drop=True)
pct_likelihood['thirty'] = thirty_pct.reset_index(drop=True)
pct_likelihood['forty'] = forty_pct.reset_index(drop=True)
pct_likelihood['fifty'] = fifty_pct.reset_index(drop=True)
pct_likelihood['sixty'] = sixty_pct.reset_index(drop=True)
pct_likelihood['seventy'] = seventy_pct.reset_index(drop=True)
pct_likelihood['eighty'] = eighty_pct.reset_index(drop=True)
pct_likelihood['ninety'] = ninety_pct.reset_index(drop=True)
pct_likelihood['one_hundred'] = one_hundred_pct.reset_index(drop=True)

# 4 Make New Visual Dataframes
YrsLeft = pd.read_csv('YrsLeft.csv')
SignedVisual = Signed.merge(YrsLeft, left_on='PLAYER', right_on='PLAYER', how='inner')
SignedVisual.loc[SignedVisual['POS'] == 1, 'POS'] = 'PG'
SignedVisual.loc[SignedVisual['POS'] == 2, 'POS'] = 'SG'
SignedVisual.loc[SignedVisual['POS'] == 3, 'POS'] = 'SF'
SignedVisual.loc[SignedVisual['POS'] == 4, 'POS'] = 'PF'
SignedVisual.loc[SignedVisual['POS'] == 5, 'POS'] = 'C'
FAsVisual = FAs
FAsVisual.loc[FAsVisual['POS'] == 1, 'POS'] = 'PG'
FAsVisual.loc[FAsVisual['POS'] == 2, 'POS'] = 'SG'
FAsVisual.loc[FAsVisual['POS'] == 3, 'POS'] = 'SF'
FAsVisual.loc[FAsVisual['POS'] == 4, 'POS'] = 'PF'
FAsVisual.loc[FAsVisual['POS'] == 5, 'POS'] = 'C'

In [2]:
# 4 Output results
print("Click here and press Shift + Enter to begin.")
# 4.1 Define function to display dataframe of Signed Players
def viewSigned(SignedTeam):
    return SignedVisual[SignedVisual['TEAM'] == SignedTeam]

# 4.2 Create a dropdown menu for user to select team
teamSigned = widgets.Dropdown(
    options = list(sorted(SignedVisual['TEAM'].unique())),
    description = 'Team: '
)

# 4.3 Using interact, display dataframes and dropdown menu
interact(viewSigned, SignedTeam = teamSigned)

Click here and press Shift + Enter to begin.


interactive(children=(Dropdown(description='Team: ', options=('ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL',…

<function __main__.viewSigned(SignedTeam)>

In [3]:
print("Click here and press Shift + Enter to lock in Team and view FAs")
display(FAsVisual)

Click here and press Shift + Enter to lock in Team and view FAs


,PLAYER_ID,PLAYER,TEAM,PREV_TEAM,TYPE,Exp,COUNTRY,DRAFT,POS,GP_2018,MIN_2018,FG3Made_2018,FTMade_2018,OREB_2018,DREB_2018,AST_2018,STL_2018,BLK_2018,PF_2018,TOV_2018,PTS_2018,GP_2017,MIN_2017,FG3Made_2017,FTMade_2017,OREB_2017,DREB_2017,AST_2017,STL_2017,BLK_2017,PF_2017,TOV_2017,PTS_2017,GP_2016,MIN_2016,FG3Made_2016,FTMade_2016,OREB_2016,DREB_2016,AST_2016,STL_2016,BLK_2016,PF_2016,TOV_2016,PTS_2016,FTMiss_2018,FTMiss_2017,FTMiss_2016,FG3Miss_2018,FG3Miss_2017,FG3Miss_2016,FG2Made_2018,FG2Miss_2018,FG2Made_2017,FG2Miss_2017,FG2Made_2016,FG2Miss_2016,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,PREV_SAL,PRED SAL
0,202329,Al-Farouq Aminu,POR,POR,UFA,9,1,8,PF,24,29.8,1.4,1.6,1.3,6.9,0.5,1.0,0.4,2.4,0.8,9.2,69.0,30.0,1.8,0.9,1.4,6.2,1.2,1.1,0.6,2.0,1.1,9.3,61.0,29.1,1.1,1.6,1.3,6.1,1.6,1.0,0.7,1.7,1.5,8.7,0.2,0.3,0.6,2.6,3.1,2.4,1.7,1.400000e+00,1.5,2.0,1.9,2.2,28.0,81,220,6957105.0,7870001
1,202692,Alec Burks,CLE,UTH,UFA,8,1,12,SG,21,17.7,0.9,2.3,0.2,2.1,1.2,0.5,0.2,1.3,1.0,9.2,64.0,16.5,0.7,1.7,0.3,2.7,1.0,0.6,0.1,1.2,0.9,7.7,42.0,15.5,0.6,1.4,0.4,2.5,0.7,0.4,0.1,1.2,0.8,6.7,0.3,0.2,0.5,1.9,1.5,1.2,2.1,2.800000e+00,2.0,2.3,1.8,2.3,27.0,78,214,11536515.0,10871497
2,203518,Alex Abrines,OKC,OKC,RFA,3,0,32,SG,20,20.3,1.5,0.4,0.2,1.4,0.8,0.7,0.2,1.8,0.5,5.9,75.0,15.1,1.1,0.5,0.3,1.2,0.4,0.5,0.1,1.7,0.3,4.7,68.0,15.5,1.4,0.6,0.3,1.0,0.6,0.5,0.1,1.7,0.5,6.0,0.1,0.1,0.1,3.1,1.8,2.2,0.5,7.000000e-01,0.4,0.6,0.6,0.8,25.0,78,200,5455236.0,6089760
3,101161,Amir Johnson,PHI,PHI,UFA,14,1,56,C,24,10.4,0.3,0.8,0.8,2.3,1.4,0.2,0.3,2.0,1.1,4.2,74.0,15.8,0.1,0.7,1.7,2.8,1.6,0.6,0.6,2.6,0.7,4.6,80.0,20.1,0.3,0.8,1.5,3.1,1.8,0.6,0.8,2.6,1.0,6.5,0.2,0.4,0.5,0.2,0.3,0.5,1.3,1.100000e+00,1.8,1.3,2.4,1.4,31.0,81,240,1512601.0,3848267
4,201229,Anthony Tolliver,MIN,MIN,UFA,12,1,0,PF,17,16.2,1.3,0.4,0.4,2.0,0.7,0.4,0.5,1.7,0.6,4.8,79.0,22.2,2.0,1.3,0.6,2.4,1.1,0.4,0.3,1.8,0.7,8.9,65.0,22.7,1.4,0.9,0.8,2.9,1.2,0.5,0.3,1.7,0.9,7.1,0.3,0.4,0.4,1.9,2.6,2.1,0.2,1.000000e-01,0.8,0.6,1.0,0.9,33.0,80,250,5750000.0,6511977
5,203085,Austin Rivers,WAS,WAS,UFA,7,1,10,SG,25,22.8,1.1,0.8,0.4,2.0,1.8,0.6,0.4,2.6,1.3,7.0,61.0,33.7,2.2,1.7,0.4,1.9,4.0,1.2,0.3,2.5,1.8,15.1,74.0,27.8,1.5,1.8,0.3,1.9,2.8,0.6,0.1,2.5,1.6,12.0,0.6,0.9,0.8,2.5,3.7,2.5,1.5,1.500000e+00,3.4,3.9,2.9,3.0,26.0,76,200,12650000.0,8366096
6,203463,Ben McLemore,SAC,SAC,UFA,6,1,7,SG,7,5.7,0.7,0.0,0.0,0.1,0.3,0.0,0.0,0.6,0.1,2.1,56.0,19.5,1.0,0.9,0.6,1.9,0.9,0.7,0.3,2.5,1.1,7.5,61.0,19.3,1.1,1.1,0.3,1.8,0.8,0.5,0.1,1.8,1.0,8.1,0.3,0.2,0.4,1.3,1.8,1.7,0.0,7.000000e-01,1.8,2.1,1.9,2.2,25.0,77,201,5460000.0,6149352
7,1626246,Boban Marjanovic,LAC,DET,UFA,4,0,0,C,18,10.8,0.0,2.6,1.8,3.0,0.6,0.3,0.8,1.7,0.8,7.9,39.0,8.7,0.0,2.0,1.2,2.5,0.6,0.3,0.3,1.3,0.9,6.0,35.0,8.4,0.0,1.3,1.3,2.4,0.3,0.2,0.3,0.7,0.3,5.5,0.7,0.5,0.4,0.1,0.0,0.0,2.7,1.400000e+00,2.0,1.8,2.1,1.7,30.0,87,290,70000.0,2411484
8,1626171,Bobby Portis,CHI,CHI,RFA,4,1,22,PF,4,28.2,0.8,1.3,1.5,8.5,2.0,0.5,1.0,3.5,1.3,10.5,73.0,22.5,1.1,1.6,2.2,4.6,1.7,0.7,0.3,1.8,1.4,13.2,64.0,15.6,0.5,0.6,1.2,3.5,0.5,0.3,0.2,1.5,0.6,6.8,1.0,0.5,0.3,2.0,2.0,1.0,3.5,4.000000e+00,4.1,3.9,2.4,2.0,23.0,83,250,2494346.0,2946832
9,202711,Bojan Bogdanovic,IND,IND,UFA,5,0,31,SF,24,31.2,2.1,2.5,0.4,3.4,1.4,0.8,0.0,1.8,1.8,15.9,80.0,30.8,1.9,2.2,0.4,3.0,1.5,0.7,0.1,1.6,1.3,14.3,81.0,25.7,1.8,2.7,0.5,3.0,1.4,0.4,0.1,1.8,1.6,13.7,0.7,0.4,0.3,2.1,2.9,3.0,3.5,2.900000e+00,3.2,2.7,2.8,2.8,29.0,80,216,10500000.0,9791896


In [4]:
# 4 Output results - User Inputs

print("Click here and press Shift + Enter to continue.")

Selected_Team = SignedVisual[SignedVisual['TEAM']==teamSigned.value]

# 4.5 Set salary cap limit and team salary given selected team
Sal_Cap = 101800000
Team_Sal = Signed[Signed['TEAM']==teamSigned.value]['SAL'].sum()

# 4.6 Loop indefinitely
while True:
# 4.7 Input a Free Agent
    FA_input = input("Select a Free Agent to sign: ")
# 4.8 If Free Agent input is "", break, if not, display a dataframe of just the selected free agent
    if (FA_input == ""):
        break
    else:
        display(FAs[FAs['PLAYER']==FA_input])
# 4.9 Input a salary offer
    Offered_Sal = int(input("How much money would you like to offer to %s? " % (FA_input)))
# 4.10 If the offer meets all requirements, print YES, if not, NO
    if (Team_Sal > Sal_Cap):
        print("NO")
    else:
        print("YES")
        # 4.11 Iterate over team salary
        Team_Sal = Team_Sal + Offered_Sal
        Selected_Player = FAsVisual[FAsVisual['PLAYER'] == FA_input]
        Selected_Player['YrsLeft'] = 1
        Selected_Player['PRED SAL'] = Offered_Sal
        Selected_Player.drop(['Exp', 'PREV_TEAM', 'TYPE'], axis=1, inplace=True)
        Selected_Player = Selected_Player.rename(index = str, columns = {'PRED SAL':'SAL'})
        Selected_Team = Selected_Team.append(Selected_Player)
        display(Selected_Team)

Click here and press Shift + Enter to continue.
Select a Free Agent to sign: Alec Burks


,PLAYER_ID,PLAYER,TEAM,PREV_TEAM,TYPE,Exp,COUNTRY,DRAFT,POS,GP_2018,MIN_2018,FG3Made_2018,FTMade_2018,OREB_2018,DREB_2018,AST_2018,STL_2018,BLK_2018,PF_2018,TOV_2018,PTS_2018,GP_2017,MIN_2017,FG3Made_2017,FTMade_2017,OREB_2017,DREB_2017,AST_2017,STL_2017,BLK_2017,PF_2017,TOV_2017,PTS_2017,GP_2016,MIN_2016,FG3Made_2016,FTMade_2016,OREB_2016,DREB_2016,AST_2016,STL_2016,BLK_2016,PF_2016,TOV_2016,PTS_2016,FTMiss_2018,FTMiss_2017,FTMiss_2016,FG3Miss_2018,FG3Miss_2017,FG3Miss_2016,FG2Made_2018,FG2Miss_2018,FG2Made_2017,FG2Miss_2017,FG2Made_2016,FG2Miss_2016,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,PREV_SAL,PRED SAL
1,202692,Alec Burks,CLE,UTH,UFA,8,1,12,SG,21,17.7,0.9,2.3,0.2,2.1,1.2,0.5,0.2,1.3,1.0,9.2,64.0,16.5,0.7,1.7,0.3,2.7,1.0,0.6,0.1,1.2,0.9,7.7,42.0,15.5,0.6,1.4,0.4,2.5,0.7,0.4,0.1,1.2,0.8,6.7,0.3,0.2,0.5,1.9,1.5,1.2,2.1,2.8,2.0,2.3,1.8,2.3,27.0,78,214,11536515.0,10871497


How much money would you like to offer to Alec Burks? 1000000
YES


,PLAYER_ID,PLAYER,TEAM,COUNTRY,DRAFT,POS,GP_2018,MIN_2018,FG3Made_2018,FTMade_2018,OREB_2018,DREB_2018,AST_2018,STL_2018,BLK_2018,PF_2018,TOV_2018,PTS_2018,GP_2017,MIN_2017,FG3Made_2017,FTMade_2017,OREB_2017,DREB_2017,AST_2017,STL_2017,BLK_2017,PF_2017,TOV_2017,PTS_2017,GP_2016,MIN_2016,FG3Made_2016,FTMade_2016,OREB_2016,DREB_2016,AST_2016,STL_2016,BLK_2016,PF_2016,TOV_2016,PTS_2016,FTMiss_2018,FTMiss_2017,FTMiss_2016,FG3Miss_2018,FG3Miss_2017,FG3Miss_2016,FG2Made_2018,FG2Miss_2018,FG2Made_2017,FG2Miss_2017,FG2Made_2016,FG2Miss_2016,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,PREV_SAL,SAL,YrsLeft
4,203458,Alex Len,ATL,0,5,C,23,20.0,0.7,1.7,1.8,3.4,0.9,0.3,0.9,3.0,1.6,9.8,69.0,20.2,0.0,2.3,2.5,5.0,1.2,0.4,0.9,2.3,1.1,8.5,77.0,20.3,0.0,1.9,2.0,4.6,0.6,0.5,1.3,3.1,1.3,8.0,0.9,1.0,0.8,1.2,0.0,0.2,3.0,2.6,3.1,2.4,3.0,2.8,25.0,85,250,4350000.0,4160000.0,1
53,1627761,DeAndre' Bembry,ATL,1,21,SF,25,23.0,0.8,1.3,0.6,3.8,2.5,1.4,0.6,2.3,1.6,8.4,26.0,17.5,0.4,0.7,0.3,2.5,1.9,0.8,0.5,1.5,1.8,5.2,38.0,9.8,0.0,0.2,0.4,1.2,0.7,0.2,0.1,0.6,0.4,2.7,0.8,0.6,0.2,1.6,0.8,0.5,2.4,2.9,1.6,2.1,1.2,0.9,24.0,78,210,1634640.0,2603982.0,3
126,1628381,John Collins,ATL,1,19,PF,10,27.4,0.4,2.1,2.1,5.8,2.4,0.1,0.5,2.8,2.2,16.9,74.0,24.1,0.2,1.8,2.4,4.9,1.3,0.6,1.1,2.9,1.4,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.7,0.0,1.7,0.4,0.0,6.8,2.6,4.0,2.8,0.0,0.0,21.0,82,235,2299080.0,2686560.0,2
150,203145,Kent Bazemore,ATL,1,0,SG,25,26.1,1.2,2.3,0.6,3.3,2.1,1.7,1.0,2.5,2.0,12.8,65.0,27.5,1.7,2.5,0.4,3.4,3.5,1.5,0.7,2.3,2.4,12.9,73.0,26.9,1.3,1.6,0.6,2.5,2.4,1.2,0.7,2.3,1.7,11.0,0.6,0.6,0.7,3.1,2.5,2.3,3.4,2.7,2.7,3.5,2.7,3.6,29.0,77,201,1889887.0,19269662.0,3
152,1628989,Kevin Huerter,ATL,1,19,SG,24,20.9,1.2,0.5,0.9,2.5,2.1,0.6,0.3,1.7,1.0,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,1.8,0.0,0.0,1.0,1.5,0.0,0.0,0.0,0.0,20.0,79,190,2250960.0,2636280.0,1
191,203101,Miles Plumlee,ATL,1,26,C,16,9.7,0.0,1.0,1.0,1.3,0.9,0.4,0.3,0.8,0.6,4.8,55.0,16.7,0.0,0.5,1.4,2.7,0.8,0.3,0.5,1.4,1.1,4.3,45.0,10.8,0.0,0.6,0.8,1.3,0.5,0.4,0.3,1.6,0.7,2.5,0.8,0.6,0.3,0.0,0.0,0.0,1.9,0.9,1.9,1.4,1.0,1.0,30.0,83,249,12500000.0,12500000.0,3
204,1629016,Omari Spellman,ATL,1,30,PF,22,18.2,0.9,1.0,1.9,2.5,1.0,0.8,0.6,1.5,0.7,6.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,1.8,0.0,0.0,1.3,1.8,0.0,0.0,0.0,0.0,21.0,81,245,1620480.0,1897800.0,1
240,1627752,Taurean Prince,ATL,1,12,SF,21,28.7,2.4,2.1,0.3,3.7,2.3,1.1,0.3,3.0,2.6,15.0,82.0,30.0,2.1,1.9,0.6,4.1,2.6,1.0,0.5,2.0,2.3,14.1,59.0,16.6,0.6,1.3,0.4,2.2,0.9,0.7,0.5,1.6,1.0,5.7,0.5,0.3,0.3,4.2,3.5,1.1,2.8,2.7,3.0,3.3,1.3,1.8,24.0,80,220,2526840.0,3481986.0,3
248,1629027,Trae Young,ATL,1,5,PG,25,29.2,1.4,3.2,0.5,2.4,7.3,0.7,0.3,1.4,4.0,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,4.3,0.0,0.0,4.2,4.9,0.0,0.0,0.0,0.0,20.0,74,180,5356440.0,6273000.0,1
1,202692,Alec Burks,CLE,1,12,SG,21,17.7,0.9,2.3,0.2,2.1,1.2,0.5,0.2,1.3,1.0,9.2,64.0,16.5,0.7,1.7,0.3,2.7,1.0,0.6,0.1,1.2,0.9,7.7,42.0,15.5,0.6,1.4,0.4,2.5,0.7,0.4,0.1,1.2,0.8,6.7,0.3,0.2,0.5,1.9,1.5,1.2,2.1,2.8,2.0,2.3,1.8,2.3,27.0,78,214,11536515.0,1000000.0,1


Select a Free Agent to sign: Alex Abrines


,PLAYER_ID,PLAYER,TEAM,PREV_TEAM,TYPE,Exp,COUNTRY,DRAFT,POS,GP_2018,MIN_2018,FG3Made_2018,FTMade_2018,OREB_2018,DREB_2018,AST_2018,STL_2018,BLK_2018,PF_2018,TOV_2018,PTS_2018,GP_2017,MIN_2017,FG3Made_2017,FTMade_2017,OREB_2017,DREB_2017,AST_2017,STL_2017,BLK_2017,PF_2017,TOV_2017,PTS_2017,GP_2016,MIN_2016,FG3Made_2016,FTMade_2016,OREB_2016,DREB_2016,AST_2016,STL_2016,BLK_2016,PF_2016,TOV_2016,PTS_2016,FTMiss_2018,FTMiss_2017,FTMiss_2016,FG3Miss_2018,FG3Miss_2017,FG3Miss_2016,FG2Made_2018,FG2Miss_2018,FG2Made_2017,FG2Miss_2017,FG2Made_2016,FG2Miss_2016,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,PREV_SAL,PRED SAL
2,203518,Alex Abrines,OKC,OKC,RFA,3,0,32,SG,20,20.3,1.5,0.4,0.2,1.4,0.8,0.7,0.2,1.8,0.5,5.9,75.0,15.1,1.1,0.5,0.3,1.2,0.4,0.5,0.1,1.7,0.3,4.7,68.0,15.5,1.4,0.6,0.3,1.0,0.6,0.5,0.1,1.7,0.5,6.0,0.1,0.1,0.1,3.1,1.8,2.2,0.5,0.7,0.4,0.6,0.6,0.8,25.0,78,200,5455236.0,6089760


How much money would you like to offer to Alex Abrines? 1000000
YES


,PLAYER_ID,PLAYER,TEAM,COUNTRY,DRAFT,POS,GP_2018,MIN_2018,FG3Made_2018,FTMade_2018,OREB_2018,DREB_2018,AST_2018,STL_2018,BLK_2018,PF_2018,TOV_2018,PTS_2018,GP_2017,MIN_2017,FG3Made_2017,FTMade_2017,OREB_2017,DREB_2017,AST_2017,STL_2017,BLK_2017,PF_2017,TOV_2017,PTS_2017,GP_2016,MIN_2016,FG3Made_2016,FTMade_2016,OREB_2016,DREB_2016,AST_2016,STL_2016,BLK_2016,PF_2016,TOV_2016,PTS_2016,FTMiss_2018,FTMiss_2017,FTMiss_2016,FG3Miss_2018,FG3Miss_2017,FG3Miss_2016,FG2Made_2018,FG2Miss_2018,FG2Made_2017,FG2Miss_2017,FG2Made_2016,FG2Miss_2016,AGE,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,PREV_SAL,SAL,YrsLeft
4,203458,Alex Len,ATL,0,5,C,23,20.0,0.7,1.7,1.8,3.4,0.9,0.3,0.9,3.0,1.6,9.8,69.0,20.2,0.0,2.3,2.5,5.0,1.2,0.4,0.9,2.3,1.1,8.5,77.0,20.3,0.0,1.9,2.0,4.6,0.6,0.5,1.3,3.1,1.3,8.0,0.9,1.0,0.8,1.2,0.0,0.2,3.0,2.6,3.1,2.4,3.0,2.8,25.0,85,250,4350000.0,4160000.0,1
53,1627761,DeAndre' Bembry,ATL,1,21,SF,25,23.0,0.8,1.3,0.6,3.8,2.5,1.4,0.6,2.3,1.6,8.4,26.0,17.5,0.4,0.7,0.3,2.5,1.9,0.8,0.5,1.5,1.8,5.2,38.0,9.8,0.0,0.2,0.4,1.2,0.7,0.2,0.1,0.6,0.4,2.7,0.8,0.6,0.2,1.6,0.8,0.5,2.4,2.9,1.6,2.1,1.2,0.9,24.0,78,210,1634640.0,2603982.0,3
126,1628381,John Collins,ATL,1,19,PF,10,27.4,0.4,2.1,2.1,5.8,2.4,0.1,0.5,2.8,2.2,16.9,74.0,24.1,0.2,1.8,2.4,4.9,1.3,0.6,1.1,2.9,1.4,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.7,0.0,1.7,0.4,0.0,6.8,2.6,4.0,2.8,0.0,0.0,21.0,82,235,2299080.0,2686560.0,2
150,203145,Kent Bazemore,ATL,1,0,SG,25,26.1,1.2,2.3,0.6,3.3,2.1,1.7,1.0,2.5,2.0,12.8,65.0,27.5,1.7,2.5,0.4,3.4,3.5,1.5,0.7,2.3,2.4,12.9,73.0,26.9,1.3,1.6,0.6,2.5,2.4,1.2,0.7,2.3,1.7,11.0,0.6,0.6,0.7,3.1,2.5,2.3,3.4,2.7,2.7,3.5,2.7,3.6,29.0,77,201,1889887.0,19269662.0,3
152,1628989,Kevin Huerter,ATL,1,19,SG,24,20.9,1.2,0.5,0.9,2.5,2.1,0.6,0.3,1.7,1.0,6.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,1.8,0.0,0.0,1.0,1.5,0.0,0.0,0.0,0.0,20.0,79,190,2250960.0,2636280.0,1
191,203101,Miles Plumlee,ATL,1,26,C,16,9.7,0.0,1.0,1.0,1.3,0.9,0.4,0.3,0.8,0.6,4.8,55.0,16.7,0.0,0.5,1.4,2.7,0.8,0.3,0.5,1.4,1.1,4.3,45.0,10.8,0.0,0.6,0.8,1.3,0.5,0.4,0.3,1.6,0.7,2.5,0.8,0.6,0.3,0.0,0.0,0.0,1.9,0.9,1.9,1.4,1.0,1.0,30.0,83,249,12500000.0,12500000.0,3
204,1629016,Omari Spellman,ATL,1,30,PF,22,18.2,0.9,1.0,1.9,2.5,1.0,0.8,0.6,1.5,0.7,6.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,1.8,0.0,0.0,1.3,1.8,0.0,0.0,0.0,0.0,21.0,81,245,1620480.0,1897800.0,1
240,1627752,Taurean Prince,ATL,1,12,SF,21,28.7,2.4,2.1,0.3,3.7,2.3,1.1,0.3,3.0,2.6,15.0,82.0,30.0,2.1,1.9,0.6,4.1,2.6,1.0,0.5,2.0,2.3,14.1,59.0,16.6,0.6,1.3,0.4,2.2,0.9,0.7,0.5,1.6,1.0,5.7,0.5,0.3,0.3,4.2,3.5,1.1,2.8,2.7,3.0,3.3,1.3,1.8,24.0,80,220,2526840.0,3481986.0,3
248,1629027,Trae Young,ATL,1,5,PG,25,29.2,1.4,3.2,0.5,2.4,7.3,0.7,0.3,1.4,4.0,15.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,4.3,0.0,0.0,4.2,4.9,0.0,0.0,0.0,0.0,20.0,74,180,5356440.0,6273000.0,1
1,202692,Alec Burks,CLE,1,12,SG,21,17.7,0.9,2.3,0.2,2.1,1.2,0.5,0.2,1.3,1.0,9.2,64.0,16.5,0.7,1.7,0.3,2.7,1.0,0.6,0.1,1.2,0.9,7.7,42.0,15.5,0.6,1.4,0.4,2.5,0.7,0.4,0.1,1.2,0.8,6.7,0.3,0.2,0.5,1.9,1.5,1.2,2.1,2.8,2.0,2.3,1.8,2.3,27.0,78,214,11536515.0,1000000.0,1


Select a Free Agent to sign: 
